Crear canva

In [1]:
# Esta celda realiza las siguientes operaciones:
# - Configura un lienzo interactivo para dibujar formas.
# - Implementa funciones para capturar dibujos y procesarlos.

from ipycanvas import Canvas # Proporciona la funcionalidad para crear lienzos interactivos en Jupyter Notebook.
from PIL import Image, ImageDraw # Se utiliza para trabajar con imágenes, como convertir de matrices a imágenes y viceversa.
import numpy as np # Es esencial para realizar operaciones matemáticas en arreglos (como las imágenes que se representan como matrices).
import os # Permite interactuar con el sistema operativo, como crear directorios.

# Se crea un lienzo blanco de 200x200 píxeles.
canvas = Canvas(width=200, height=200, background_color="white", sync_image_data = True)
# La opción sync_image_data = True asegura que los cambios en el lienzo se reflejen en la representación de la imagen.

# Función para capturar el dibujo como imagen
def get_drawing(): # Obtiene los datos de la imagen del lienzo.
    # Convierte la imagen del lienzo a una matriz NumPy y luego a imagen de escala de grises    
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    # Convertir la imagen a escala de grises (L)
    img = img.convert("L")
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    # Convertir la imagen en escala de grises a una matriz NumPy
    return np.array(img)

# Función para guardar el dibujo
def save_drawing(class_name, count): # Crea un directorio para almacenar las imágenes de la clase especificada (por ejemplo, "0", "1", "2" para dígitos).
    # Crear directorio si no existe
    os.makedirs(f"data/{class_name}", exist_ok=True) # exist_ok=True evita errores si el directorio ya existe.
    # Obtiene el dibujo del lienzo utilizando get_drawing().
    img = get_drawing()
    
    # Guardar la imagen en el directorio especificado
    filepath = f"data/{class_name}/{count}.png"
    Image.fromarray(img).save(filepath)
    print(f"Dibujo guardado en: {filepath}")


# Variable para almacenar la última posición
last_x, last_y = None, None

# Función para dibujar en el lienzo
def on_mouse_down(x, y): 
    global last_x, last_y
    canvas.fill_style = "black"
    last_x, last_y = x, y  # Guardar la posición inicial cuando se presiona el botón del mouse

def on_mouse_move(x, y): # Se definen funciones para manejar los eventos de clic, movimiento y liberación del mouse.
    global last_x, last_y
    if last_x is not None and last_y is not None:
        canvas.stroke_style = "black"
        canvas.line_width = 5
        canvas.begin_path()
        canvas.move_to(last_x, last_y)
        canvas.line_to(x, y)
        canvas.stroke()
        last_x, last_y = x, y  # Actualizar la posición de la última coordenada

def on_mouse_up(x, y):
    global last_x, last_y
    last_x, last_y = None, None  # Resetear cuando se suelta el mouse

# Asignar los eventos de mouse al lienzo
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

# Mostrar el lienzo
display(canvas)


Canvas(height=200, sync_image_data=True, width=200)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple neural network model (example)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Example input size for MNIST
        self.fc2 = nn.Linear(128, 10)    # Example output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleModel()  # Define the model here

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the images
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Create DataLoader

criterion = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters())  # Example optimizer

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Ensure device is defined
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Ensure you have a device defined (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Train the model
train_model(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 0.3449, Accuracy: 90.57%
Epoch 2/5, Loss: 0.1569, Accuracy: 95.47%
Epoch 3/5, Loss: 0.1094, Accuracy: 96.87%
Epoch 4/5, Loss: 0.0836, Accuracy: 97.51%
Epoch 5/5, Loss: 0.0663, Accuracy: 98.06%


In [5]:
# Se guarda imagen en un directorio específico.
save_drawing("square", 5)

Dibujo guardado en: data/square/5.png


Generar Imágenes

In [9]:
# Esta celda realiza las siguientes operaciones:
# - Genera datos sintéticos de varias formas geométricas (círculos, cuadrados, etc.).
# - Guarda estas imágenes en una estructura de carpetas.

# Generar formas sintéticas

def generate_synthetic_data(class_name, count): # Esta función genera y guarda imágenes de formas geométricas básicas en blanco y negro.
    os.makedirs(f"data/{class_name}", exist_ok=True)
    for i in range(count):
        # Crear una imagen en blanco
        img = Image.new("L", (28, 28), "white") # Imagen de  28X28 píxeles en escala de grises "L"
        draw = ImageDraw.Draw(img) # Crea un objeto ImageDraw.Draw para dibujar en la imagen.

        # Nombre de la clase de la forma geométrica a generar. Debe ser "circle", "square", "triangle" o "star"
        if class_name == "circle":
            draw.ellipse((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "square":
            draw.rectangle((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "triangle":
            draw.polygon([(14, 5), (5, 23), (23, 23)], outline="black", fill="black")
        elif class_name == "star":
            draw.polygon([(14, 5), (10, 20), (5, 14), (23, 14), (18, 20)], outline="black", fill="black")

        # Guardar la imagen
        filepath = f"data/{class_name}/{i}.png" # Ruta del archivo 
        img.save(filepath) # Acá se guarda la imagen en la ruta 
        #print(f"Dibujo sintético guardado en: {filepath}")

# Generar 100 imágenes sintéticas por clase
generate_synthetic_data("circle", 100)
generate_synthetic_data("square", 100)
generate_synthetic_data("triangle", 100)
generate_synthetic_data("star", 100)


Cargar imágenes en el Dataset

In [11]:
# Importar los módulos necesarios
import os
from PIL import Image, ImageDraw  # Importar las clases necesarias de PIL

# Esta celda realiza las siguientes operaciones:
# - Genera datos sintéticos de varias formas geométricas (círculos, cuadrados, etc.).
# - Guarda estas imágenes en una estructura de carpetas.

# Generar formas sintéticas
def generate_synthetic_data(class_name, count):  # Esta función genera y guarda imágenes de formas geométricas básicas en blanco y negro.
    os.makedirs(f"data/{class_name}", exist_ok=True)
    for i in range(count):
        # Crear una imagen en blanco
        img = Image.new("L", (28, 28), "white")  # Imagen de 28X28 píxeles en escala de grises "L"
        draw = ImageDraw.Draw(img)  # Crea un objeto ImageDraw.Draw para dibujar en la imagen.

        # Nombre de la clase de la forma geométrica a generar. Debe ser "circle", "square", "triangle" o "star"
        if class_name == "circle":
            draw.ellipse((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "square":
            draw.rectangle((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "triangle":
            draw.polygon([(14, 5), (5, 23), (23, 23)], outline="black", fill="black")
        elif class_name == "star":
            draw.polygon([(14, 5), (10, 20), (5, 14), (23, 14), (18, 20)], outline="black", fill="black")

        # Guardar la imagen
        filepath = f"data/{class_name}/{i}.png"  # Ruta del archivo 
        img.save(filepath)  # Acá se guarda la imagen en la ruta 
        # print(f"Dibujo sintético guardado en: {filepath}")

# Generar 100 imágenes sintéticas por clase
generate_synthetic_data("circle", 100)
generate_synthetic_data("square", 100)
generate_synthetic_data("triangle", 100)
generate_synthetic_data("star", 100)

In [9]:
# Esta celda realiza las siguientes operaciones:
# - Define una clase personalizada para cargar imágenes en un dataset.
# - Aplica transformaciones como normalización y conversión a tensor.

import os
import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class DrawingDataset(Dataset): # root_dir es la ruta del directorio raíz que contiene las imágenes del dataset.
    def __init__(self, root_dir, transform=None): # Metodo que inicializa el dataset, verifica exisitencia de la imagen 
        self.root_dir = root_dir
        self.transform = transform # Del objeto de transforms de torchvision para transformaciones que se aplicarán a las imágenes cargadas.
        self.filepaths = glob.glob(os.path.join(root_dir, "*", "*.png")) # Lista de rutas de las imágenes en el dataset

        if not self.filepaths:
            raise ValueError(f"No se encontraron imágenes en {root_dir}. Verifica la estructura del directorio.")
            
        # Lista de etiquetas correspondientes a cada imagen, basadas en el nombre del directorio padre
        self.labels = [os.path.basename(os.path.dirname(path)) for path in self.filepaths]
        # Diccionario que mapea cada etiqueta única a un índice entero.
        self.label_to_idx = {label: idx for idx, label in enumerate(set(self.labels))}

    # Devuelve el tamaño del dataset (número total de imágenes).
    def __len__(self):
        return len(self.filepaths)

    # Obtiene una imagen y su etiqueta correspondiente a un índice específico
    def __getitem__(self, idx):
            # Abrir imagen y convertirla a escala de grises
            img = Image.open(self.filepaths[idx]).convert("L")  # Convertir a escala de grises (1 canal)
            label = self.label_to_idx[self.labels[idx]] # Obtiene la etiqueta del diccionario
            
            # Aplicar transformaciones
            if self.transform:
                img = self.transform(img)
            else:
                # Si no hay transformaciones, convierte la imagen a un tensor usando
                img = transforms.ToTensor()(img)
            
            return img, label # Devuelve una tupla con la imagen (como tensor) y su etiqueta (como entero).
# Transformaciones para normalizar imágenes
transform = transforms.Compose([
    transforms.Resize((28, 28)),       # Asegurar tamaño 28x28
    transforms.ToTensor(),            # Convertir a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalizar
])

# Cargar el dataset
dataset = DrawingDataset(root_dir="data", transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


Paso 3: Crear y Entrenar el Modelo CNN Definimos un modelo básico en PyTorch.

In [13]:
# Esta celda realiza las siguientes operaciones:
# - Define un modelo básico de PyTorch y entrena una red neuronal CNN.
# - Imprime estadísticas como pérdida y precisión por época.

import torch.nn.functional as F


    # Verificar si GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}") # Imprime el dispositivo utilizado para el entrenamiento.

# Enviar el modelo al dispositivo
model.to(device)

# Función para entrenar el modelo CNN.
# Dataloader: Cargador de datos (DataLoader) creado a partir del dataset DrawingDataset.
# Criterion: Función de pérdida para evaluar el error del modelo (definido más adelante).
# Optimizer: Optimizador para ajustar los pesos del modelo durante el entrenamiento.
# Epochs (opcional, int): Número de épocas para entrenar el modelo (valor por defecto 5).
    
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()  # Modo entrenamiento
    for epoch in range(epochs):
        # Variable para acumular la pérdida total durante una época.
        running_loss = 0.0
        # Variable para acumular el número de predicciones correctas.
        correct = 0
        # total = 0 # Variable para acumular el número total de imágenes en una época.
        total = 0

        # Envía las imágenes y etiquetas al dispositivo seleccionado (GPU o CPU) para el procesamiento.
        for images, labels in dataloader:
            # Enviar imágenes y etiquetas al dispositivo
            images, labels = images.to(device), labels.to(device)

            # Pone a cero los gradientes del optimizador antes de cada paso de retropropagación.
            # Forward
            optimizer.zero_grad()
            outputs = model(images) #  Realiza una pasada hacia adelante por el modelo para obtener las salidas (predicciones)
            loss = criterion(outputs, labels) # Calcula la pérdida entre las salidas del modelo y las etiquetas reales utilizando la función de pérdida.
            
            # Backward y optimización
            loss.backward() # Calcula los gradientes de la pérdida con respecto a los pesos del modelo.
            optimizer.step() # Actualiza los pesos del modelo utilizando el optimizador y los gradientes calculados.
            
            # Estadísticas
            running_loss += loss.item() # Acumula la pérdida del lote actual.
            _, predicted = torch.max(outputs, 1) # Obtiene las clases predichas (índice de la clase con mayor probabilidad).
            total += labels.size(0) # Contabiliza el número de imágenes en el lote actual.
            correct += (predicted == labels).sum().item() # Contabiliza el número de predicciones correctas en el lote actual.

        accuracy = 100 * correct / total #  Calcula la precisión del modelo para la época actual.
        # Imprime el número de la época actual, la pérdida promedio y la precisión para la época.
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)


Usando dispositivo: cpu
Epoch 1/5, Loss: 0.0544, Accuracy: 98.37%
Epoch 2/5, Loss: 0.0448, Accuracy: 98.67%
Epoch 3/5, Loss: 0.0357, Accuracy: 98.91%
Epoch 4/5, Loss: 0.0298, Accuracy: 99.10%
Epoch 5/5, Loss: 0.0254, Accuracy: 99.27%


Entrenamos el modelo con los datos capturados.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network model (example)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Example input size for MNIST
        self.fc2 = nn.Linear(128, 10)    # Example output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleModel()  # Define the model here

# Assuming 'dataloader', 'criterion', and 'optimizer' are already defined
# Example definitions (you would replace these with your actual data and settings)
dataloader = ...  # Your DataLoader here
criterion = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters())  # Example optimizer

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

TypeError: 'ellipsis' object is not iterable

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define a simple neural network model (example)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Example input size for MNIST
        self.fc2 = nn.Linear(128, 10)    # Example output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleModel()  # Define the model here

# Example data for demonstration (replace with your actual data)
# Here we create random data for 100 samples, each with 784 features, and random labels
dummy_images = torch.randn(100, 784)  # 100 samples of 784 features
dummy_labels = torch.randint(0, 10, (100,))  # 100 random labels for 10 classes

# Create a DataLoader
dataset = TensorDataset(dummy_images, dummy_labels)  # Combine images and labels
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # Create DataLoader

criterion = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters())  # Example optimizer

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Train the model
train_model(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 2.3568, Accuracy: 12.00%
Epoch 2/5, Loss: 1.3892, Accuracy: 91.00%
Epoch 3/5, Loss: 0.8221, Accuracy: 100.00%
Epoch 4/5, Loss: 0.4583, Accuracy: 100.00%
Epoch 5/5, Loss: 0.2399, Accuracy: 100.00%


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple neural network model (example)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Example input size for MNIST
        self.fc2 = nn.Linear(128, 10)    # Example output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleModel()  # Define the model here

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the images
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Create DataLoader

criterion = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters())  # Example optimizer

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Ensure device is defined
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Ensure you have a device defined (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Train the model
train_model(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 0.3434, Accuracy: 90.54%
Epoch 2/5, Loss: 0.1616, Accuracy: 95.30%
Epoch 3/5, Loss: 0.1112, Accuracy: 96.76%
Epoch 4/5, Loss: 0.0840, Accuracy: 97.56%
Epoch 5/5, Loss: 0.0664, Accuracy: 98.02%


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a simple neural network model (example)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Example input size for MNIST
        self.fc2 = nn.Linear(128, 10)    # Example output size for 10 classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleModel()  # Define the model here

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten the images
])

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # Create DataLoader

criterion = nn.CrossEntropyLoss()  # Example loss function
optimizer = optim.Adam(model.parameters())  # Example optimizer

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Ensure device is defined
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Ensure you have a device defined (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Train the model
train_model(model, dataloader, criterion, optimizer, epochs=5)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:01<00:00, 9.41MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 256kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 3.46MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<?, ?B/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch 1/5, Loss: 0.3452, Accuracy: 90.52%
Epoch 2/5, Loss: 0.1612, Accuracy: 95.36%
Epoch 3/5, Loss: 0.1131, Accuracy: 96.70%
Epoch 4/5, Loss: 0.0849, Accuracy: 97.51%
Epoch 5/5, Loss: 0.0667, Accuracy: 97.99%


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Capa totalmente conectada (10 entradas, 2 salidas)

    def forward(self, x):
        return self.fc(x)

# Inicializa el modelo
model = SimpleModel()  # Asegúrate de que el modelo esté definido antes de usarlo

# Definición del dataloader, criterion y optimizer (ejemplo)
dataloader = ...  # Aquí deberías definir tu dataloader
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

TypeError: 'ellipsis' object is not iterable

In [ ]:
import torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Capa totalmente conectada (10 entradas, 2 salidas)

    def forward(self, x):
        return self.fc(x)

# Inicializa el modelo
model = SimpleModel()  # Asegúrate de que el modelo esté definido antes de usarlo

# Definición del dataloader, criterion y optimizer (ejemplo)
dataloader = ...  # Aquí deberías definir tu dataloader
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Definición de la función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()  # Establece el modelo en modo de entrenamiento
    for epoch in range(epochs):
        for inputs, labels in dataloader:  # Itera sobre el dataloader
            optimizer.zero_grad()  # Limpia los gradientes
            outputs = model(inputs)  # Realiza la predicción
            loss = criterion(outputs, labels)  # Calcula la pérdida
            loss.backward()  # Calcula los gradientes
            optimizer.step()  # Actualiza los parámetros del modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')  # Imprime la pérdida por época

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Capa totalmente conectada (10 entradas, 2 salidas)

    def forward(self, x):
        return self.fc(x)

# Inicializa el modelo
model = SimpleModel()  # Asegúrate de que el modelo esté definido antes de usarlo

# Crear datos de ejemplo
inputs = torch.randn(100, 10)  # 100 muestras, 10 características
labels = torch.randint(0, 2, (100,))  # 100 etiquetas (0 o 1)

# Definición del dataloader
dataset = TensorDataset(inputs, labels)  # Combina inputs y labels en un dataset
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # Crea el dataloader

criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Definición de la función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()  # Establece el modelo en modo de entrenamiento
    for epoch in range(epochs):
        for inputs, labels in dataloader:  # Itera sobre el dataloader
            optimizer.zero_grad()  # Limpia los gradientes
            outputs = model(inputs)  # Realiza la predicción
            loss = criterion(outputs, labels)  # Calcula la pérdida
            loss.backward()  # Calcula los gradientes
            optimizer.step()  # Actualiza los parámetros del modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')  # Imprime la pérdida por época

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 1.1295737028121948
Epoch 2/5, Loss: 0.7443002462387085
Epoch 3/5, Loss: 0.6539721488952637
Epoch 4/5, Loss: 0.7315527200698853
Epoch 5/5, Loss: 0.8042091131210327


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Capa totalmente conectada (10 entradas, 2 salidas)

    def forward(self, x):
        return self.fc(x)

# Inicializa el modelo
model = SimpleModel()  # Asegúrate de que el modelo esté definido antes de usarlo

# Definición del dataloader, criterion y optimizer (ejemplo)
dataloader = ...  # Aquí deberías definir tu dataloader
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Definición de la función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()  # Establece el modelo en modo de entrenamiento
    for epoch in range(epochs):
        for inputs, labels in dataloader:  # Itera sobre el dataloader
            optimizer.zero_grad()  # Limpia los gradientes
            outputs = model(inputs)  # Realiza la predicción
            loss = criterion(outputs, labels)  # Calcula la pérdida
            loss.backward()  # Calcula los gradientes
            optimizer.step()  # Actualiza los parámetros del modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')  # Imprime la pérdida por época

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 2)  # Capa totalmente conectada (10 entradas, 2 salidas)

    def forward(self, x):
        return self.fc(x)

# Inicializa el modelo
model = SimpleModel()  # Asegúrate de que el modelo esté definido antes de usarlo

# Ejemplo de datos aleatorios para el DataLoader
# Genera 100 muestras de entrada (10 características cada una) y 100 etiquetas
inputs = torch.randn(100, 10)  # 100 muestras, 10 características
labels = torch.randint(0, 2, (100,))  # 100 etiquetas (0 o 1)

# Definición del dataloader, criterion y optimizer (ejemplo)
dataset = TensorDataset(inputs, labels)  # Crea un dataset a partir de los datos
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # Crea un DataLoader
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Definición de la función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, epochs):
    model.train()  # Establece el modelo en modo de entrenamiento
    for epoch in range(epochs):
        for inputs, labels in dataloader:  # Itera sobre el dataloader
            optimizer.zero_grad()  # Limpia los gradientes
            outputs = model(inputs)  # Realiza la predicción
            loss = criterion(outputs, labels)  # Calcula la pérdida
            loss.backward()  # Calcula los gradientes
            optimizer.step()  # Actualiza los parámetros del modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')  # Imprime la pérdida por época

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

Epoch 1/5, Loss: 0.7226508855819702
Epoch 2/5, Loss: 0.8095153570175171
Epoch 3/5, Loss: 0.5706993341445923
Epoch 4/5, Loss: 0.8168092966079712
Epoch 5/5, Loss: 0.5329379439353943


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(784, 10)  # Ejemplo para un modelo de clasificación de imágenes (como MNIST)

    def forward(self, x):
        return self.fc(x)

# Crear una instancia del modelo
model = SimpleModel()  # Definimos el modelo aquí

# Definición del criterio y optimizador
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Asegúrate de que 'dataloader' y 'device' estén definidos antes de llamar a 'train_model'
# Por ejemplo:
# dataloader = ...  # Tu DataLoader aquí
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)  # Mover el modelo al dispositivo

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definición de un modelo simple (ejemplo)
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(784, 10)  # Ejemplo para un modelo de clasificación de imágenes (como MNIST)

    def forward(self, x):
        return self.fc(x)

# Crear una instancia del modelo
model = SimpleModel()  # Definimos el modelo aquí

# Definición del criterio y optimizador
criterion = nn.CrossEntropyLoss()  # Función de pérdida
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizador

# Definición de la función de entrenamiento
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()  # Poner el modelo en modo de entrenamiento
    for epoch in range(epochs):
        for inputs, labels in dataloader:  # Iterar sobre el DataLoader
            optimizer.zero_grad()  # Reiniciar los gradientes
            outputs = model(inputs)  # Hacer una predicción
            loss = criterion(outputs, labels)  # Calcular la pérdida
            loss.backward()  # Retropropagar la pérdida
            optimizer.step()  # Actualizar los parámetros del modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')  # Imprimir la pérdida por época

# Asegúrate de que 'dataloader' y 'device' estén definidos antes de llamar a 'train_model'
# Por ejemplo:
# dataloader = ...  # Tu DataLoader aquí
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)  # Mover el modelo al dispositivo

# Entrenar el modelo
# train_model(model, dataloader, criterion, optimizer, epochs=5)  # Descomentar esta línea cuando 'dataloader' esté definido

In [ ]:
import torch

In [ ]:
# Mostrar el lienzo
from IPython.display import display  # Importar la función display

# Crear un lienzo (canvas) de ejemplo
import matplotlib.pyplot as plt

# Crear un lienzo (canvas) usando matplotlib
fig, ax = plt.subplots()
canvas = fig.canvas  # Definir el objeto canvas

# Mostrar el lienzo
display(canvas)  # Mostrar el lienzo

In [ ]:
# Esta celda realiza las siguientes operaciones:
# - Utiliza un modelo previamente entrenado para predecir la clase de un dibujo en el lienzo.
# - Limpia el lienzo después de realizar una predicción.

import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# Crear la transformación que se debe aplicar al dibujo
transform = transforms.Compose([
    transforms.ToTensor(),                  # Convierte a tensor
    transforms.Normalize((0.5,), (0.5,))    # Normaliza (ajusta según las necesidades del modelo)
])

# Función para obtener la imagen desde el lienzo
def get_drawing():
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    img = img.convert("L")  # Convertir a escala de grises
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    return np.array(img)  # Retorna como un array NumPy

# Función para predecir usando el modelo
def predict_drawing(model, dataset):
    # Obtener el dibujo actual del lienzo
    img = get_drawing()
    
    # Preprocesar la imagen
    img_tensor = transform(Image.fromarray(img)).unsqueeze(0)  # Agregar dimensión batch
    
    # Enviar la imagen al modelo para predicción
    model.eval()  # Establecer el modelo en modo evaluación
    with torch.no_grad():  # Desactivar gradientes para predicción
        output = model(img_tensor)  # Obtener las predicciones
        pred = torch.argmax(output, dim=1).item()  # Obtener la clase predicha
    
    # Mapear la predicción a la etiqueta correspondiente
    label = list(dataset.label_to_idx.keys())[list(dataset.label_to_idx.values()).index(pred)]
    print(f"Predicción: {label}")

# Ejemplo de uso después de dibujar algo en el lienzo:
predict_drawing(model, dataset)

#Limpiar el canvas después de la prediccion
canvas.clear()

